In [8]:
import pandas as pd

In [9]:
import random
import pandas as pd

# Função para gerar tentativas com condições específicas
def gerar_tentativa(num_tentativa):
    tentativa = []
    for _ in range(10):  # 10 questões no quiz
        # Na primeira tentativa, definimos 70% de chance de erro e um tempo mais longo
        if num_tentativa == 1:
            resultado = random.choices([True, False], weights=[0.3, 0.7])[0]
            tempo = random.randint(20, 40)  # Tempo aleatório entre 20 e 40 segundos
        # Nas duas últimas tentativas, garantimos mais acertos do que erros
        elif num_tentativa >= 3:
            resultado = random.choices([True, False], weights=[0.8, 0.2])[0]
            tempo = random.randint(10, 30)  # Tempo aleatório entre 10 e 30 segundos
        # Tentativas intermediárias com distribuição uniforme
        else:
            resultado = random.choice([True, False])  
            tempo = random.randint(10, 30)  # Tempo aleatório entre 10 e 30 segundos
        tentativa.append((resultado, tempo))
    return pd.Series(tentativa)

# Dicionário para armazenar as respostas dos usuários
respostas = {}

# Gerar 100 novos usuários com 4 tentativas diferentes cada
for i in range(1, 101):
    user_id = f'User_{i}'
    tentativas_usuario = {}
    for j in range(1, 5):
        nome_tentativa = f'Tentativa_{j}'
        tentativa = gerar_tentativa(j)
        tentativas_usuario[nome_tentativa] = tentativa
    respostas[user_id] = tentativas_usuario

# Criar DataFrame com os resultados das tentativas dos usuários
data = []
for user_id, tentativas_usuario in respostas.items():
    for tentativa, respostas_tentativa in tentativas_usuario.items():
        data.append([user_id, tentativa, respostas_tentativa])

# Criar DataFrame a partir dos dados coletados
df = pd.DataFrame(data, columns=['User_ID', 'Tentativa', 'Respostas'])


In [10]:
class Quiz:
    def __init__(self, nome_quiz, total_questoes):
        self.nome_quiz = nome_quiz
        self.total_questoes = total_questoes
        self.acertos = 0
        self.erros = 0
        self.duracao_total = 0
        self.user = ''
        self.tentativa = 0
        self.resultados = {}
        self.percentual_acertos = 0
        self.media_tempo_resposta = 0
        self.mensagem = ''

    
    # Função responsável por contabilizar os acertos, os erros e o tempo toda da partida avaliar_desempenho(self)
    def respostas (self, user='',  tentativa=0, resposta_correta=[], tempo_resposta=[]):

        self.tentativa = tentativa
        self.user = user
        self.duracao_total = sum(tempo_resposta)

        for resposta in resposta_correta:
            if resposta:
                self.acertos += 1
            else:
                self.erros += 1
        
        # Calculando desempenho
        self.media_tempo_resposta = self.duracao_total / self.total_questoes
        self.percentual_acertos   = (self.acertos / self.total_questoes) * 100

        # Avaliando o resultado do usuário
        if self.percentual_acertos >= 70 and self.media_tempo_resposta <= 20:
            mensagem = "APROVADO"
        else:
            mensagem = "REPROVADO"

        self.mensagem = mensagem
        

    def armazenando_dados (self):

        if self.tentativa != 0:
            self.resultados[self.user,f"Tentativa {self.tentativa}"]= {
                    'Quiz':                      self.nome_quiz,
                    'Acertos':                   self.acertos,
                    'Erros':                     self.erros,
                    'Acertos(%)':                self.percentual_acertos,
                    'Média tempo resposta(s)':   self.media_tempo_resposta,
                    'Duração do quiz(min)':      (self.duracao_total/60),
                    'Status':                    self.mensagem
                }
            
            df = pd.DataFrame(self.resultados).T

            self.acertos = 0
            self.erros = 0
            self.percentual_acertos = 0
            self.media_tempo_resposta = 0
            self.duracao_total = 0
            self.mensagem = 0

            return df
            

In [11]:
# INPUT INICIAL
nome_quiz = 'Exame - Hemograma'
qtd       = 10

quiz = Quiz(nome_quiz,qtd)

In [12]:
# Função responsável por  filtrar as informações do DataFrame
def analise_tentativa (data, tentativa):

        for user in  data['User_ID']:
                respostas = []
                tempo     = []
                # Filtrando o DataFrame
                data_filtrado =  data.loc[(data['User_ID'] == user) & (data['Tentativa'] == f'Tentativa_{tentativa}'), 'Respostas'].iloc[0]
                for questao in range(quiz.total_questoes):
                                
                        respostas.append(data_filtrado[questao][0])
                        tempo.append(data_filtrado[questao][1])
                        
                quiz.armazenando_dados()        
                quiz.respostas(user, tentativa, respostas, tempo)


In [13]:

def consulta(df):

    for vez in range(1, 5):
        analise_tentativa(df, vez)
    
    data_user = quiz.armazenando_dados()
    df_user  = pd.DataFrame(data_user)

    #  Mostrando as informações coletadas sobre o usuário
    return df_user
    

In [14]:
consulta(df).to_excel('dados_geral.xlsx', index=False)